VacationPy

In [1]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#Import API key
from config import gkey

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
city_data_file = "../WeatherPy/cities.csv"

city_df = pd.read_csv(city_data_file)

city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,rikitea,61,PF,1586749115,77,-23.12,-134.97,77.70,18.05
1,1,arman,100,RU,1586749115,94,59.70,150.17,25.32,4.07
2,2,port alfred,12,ZA,1586749116,79,-33.59,26.89,57.99,3.00
3,3,ushuaia,90,AR,1586748920,81,-54.80,-68.30,42.80,11.41
4,4,mataura,99,NZ,1586748923,83,-46.19,168.86,45.00,11.99


Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key.
gmaps.configure(api_key=gkey)

#Convert Humidity and remove missing information
city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)

#Store Latitude and Longitude locations
locations = city_df[["Lat", "Lng"]].astype(float)

#Create a humidity heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [4]:
#Narrow down cities to only show ideal weather
narrow_city_df = city_df

#Max temperature of 80, but higher than 70
narrow_city_df = narrow_city_df.loc[(narrow_city_df["Max Temp"] < 80) & (narrow_city_df["Max Temp"] > 70)]

#Wind speeds less than 10 mph
narrow_city_df = narrow_city_df.loc[narrow_city_df["Wind Speed"] < 10]

#Zero cloudiness
narrow_city_df = narrow_city_df.loc[narrow_city_df["Cloudiness"] == 0]

#Drop any rows without information
narrow_city_df = narrow_city_df.dropna()

narrow_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,30,linxi,0,CN,1586749123,19,39.70,118.43,71.71,3.33
129,143,beloha,0,MG,1586749152,67,-25.17,45.05,72.19,6.06
148,165,vangaindrano,0,MG,1586749157,91,-23.35,47.60,70.61,1.32
194,214,namibe,0,AO,1586749169,76,-15.20,12.15,76.37,2.77
258,284,farafangana,0,MG,1586749186,85,-22.82,47.83,73.42,2.42
288,316,niquero,0,CU,1586749194,82,20.04,-77.58,77.72,7.96


Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [5]:
#Create the Hotel database
hotel_df = city_df

#Parameter dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    #Change location while leaving original params
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Create URL and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    #Show results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: rikitea.
Closest hotel is Pension Maro'i.
------------
Retrieving Results for Index 1: arman.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: port alfred.
Closest hotel is The Halyards Hotel.
------------
Retrieving Results for Index 3: ushuaia.
Closest hotel is Albatros Hotel.
------------
Retrieving Results for Index 4: mataura.
Closest hotel is Ellie's Villa.
------------
Retrieving Results for Index 5: severo-kurilsk.
Closest hotel is Kak Doma.
------------
Retrieving Results for Index 7: hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 8: bengkulu.
Closest hotel is Grage Hotel Bengkulu.
------------
Retrieving Results for Index 9: duma.
Closest hotel is 永寿服务区-宾馆.
------------
Retrieving Results for Index 10: bredasdorp.
Closest hotel is Bredasdorp Country Manor.
------------
Retrieving Results for Index 11: codrington.
Closest hotel is Codrington Gardens Bed And Breakfast.
---------

Closest hotel is LE SAMANA.
------------
Retrieving Results for Index 91: novoagansk.
Missing field/result... skipping.
------------
Retrieving Results for Index 92: lorengau.
Closest hotel is Lorengau Harbourside Hotel.
------------
Retrieving Results for Index 93: ternate.
Closest hotel is Grand Dafam Bela Ternate.
------------
Retrieving Results for Index 94: corbelia.
Closest hotel is Peroza Hotel e Restaurante.
------------
Retrieving Results for Index 95: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 96: avarua.
Closest hotel is Paradise Inn.
------------
Retrieving Results for Index 97: teguise.
Closest hotel is B&B La Casona de Nazaret.
------------
Retrieving Results for Index 98: turukhansk.
Closest hotel is ООО "Туруханская пушнина".
------------
Retrieving Results for Index 99: kihei.
Closest hotel is Maui Sunset.
------------
Retrieving Results for Index 100: troitskoye.
Missing field/result... skipping.
------------
Retrieving Resu

Missing field/result... skipping.
------------
Retrieving Results for Index 178: faanui.
Closest hotel is Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 179: upernavik.
Closest hotel is Gina's Guesthouse - Upernavik.
------------
Retrieving Results for Index 180: khani.
Closest hotel is კურორტი ზეკარი/Resort Zekari.
------------
Retrieving Results for Index 181: otane.
Closest hotel is Ludlow Farmstay.
------------
Retrieving Results for Index 182: acapulco.
Closest hotel is HS Hotsson Smart Acapulco.
------------
Retrieving Results for Index 183: carauari.
Closest hotel is Pousada Lobo.
------------
Retrieving Results for Index 184: pemangkat.
Closest hotel is Kost H.Syamsuddin.
------------
Retrieving Results for Index 185: villafranca de los barros.
Closest hotel is HOTEL ACOSTA CIUDAD DE LA MUSICA.
------------
Retrieving Results for Index 186: tuktoyaktuk.
Closest hotel is End of The Road Ltd..
------------
Retrieving Results for Index 187: ust-omchu

Closest hotel is Urban Style by Front One.
------------
Retrieving Results for Index 263: saquarema.
Closest hotel is Hotel Fazenda Sol e Lua.
------------
Retrieving Results for Index 264: saint-philippe.
Closest hotel is Chambres d'hôte "La Trinité".
------------
Retrieving Results for Index 265: vilyuysk.
Closest hotel is Gostinitsa Viliuisk.
------------
Retrieving Results for Index 266: port augusta.
Closest hotel is Majestic Oasis Apartments.
------------
Retrieving Results for Index 267: airai.
Closest hotel is Juvinal Martin.
------------
Retrieving Results for Index 268: sept-iles.
Closest hotel is Comfort Inn.
------------
Retrieving Results for Index 269: katsuura.
Closest hotel is Katsuura Hotel Mikazuki.
------------
Retrieving Results for Index 270: sao paulo de olivenca.
Missing field/result... skipping.
------------
Retrieving Results for Index 271: alofi.
Closest hotel is Taloa Heights.
------------
Retrieving Results for Index 272: linda.
Closest hotel is Comfort Suit

Closest hotel is Kabash Inn.
------------
Retrieving Results for Index 349: merritt island.
Missing field/result... skipping.
------------
Retrieving Results for Index 350: umm lajj.
Closest hotel is Tanal Hotel Apartment.
------------
Retrieving Results for Index 351: ponta do sol.
Closest hotel is Hotel do Campo.
------------
Retrieving Results for Index 352: am timan.
Missing field/result... skipping.
------------
Retrieving Results for Index 353: santiago.
Closest hotel is Hotel Panamericano.
------------
Retrieving Results for Index 354: lakselv.
Closest hotel is Lakselv Hotel.
------------
Retrieving Results for Index 355: visnes.
Closest hotel is Park Inn Haugesund Airport.
------------
Retrieving Results for Index 356: suclayin.
Closest hotel is Costa Pacifica.
------------
Retrieving Results for Index 357: shelburne.
Closest hotel is The Retreat at the Farm.
------------
Retrieving Results for Index 359: savannah bight.
Closest hotel is Villa on Dunbar Rock.
------------
Retri

Closest hotel is Lord Grey Guest House.
------------
Retrieving Results for Index 436: khotyn.
Closest hotel is Khotyn, Hotelʹ.
------------
Retrieving Results for Index 437: freeport.
Closest hotel is The Freeport Inn and Marina.
------------
Retrieving Results for Index 438: vila velha.
Closest hotel is Hotel Vitória Palace.
------------
Retrieving Results for Index 439: yarmouth.
Closest hotel is The Inn At Cape Cod.
------------
Retrieving Results for Index 440: kieta.
Missing field/result... skipping.
------------
Retrieving Results for Index 441: wajir.
Closest hotel is Al- Amin lodge.
------------
Retrieving Results for Index 442: beidao.
Closest hotel is Shell Tianshui Maiji District Erma Road Walking Street Hotel.
------------
Retrieving Results for Index 443: balakhninskiy.
Missing field/result... skipping.
------------
Retrieving Results for Index 444: tura.
Closest hotel is Hotel RIKMAN Continental.
------------
Retrieving Results for Index 445: velyka bahachka.
Closest hot

Closest hotel is Havsdalsgrenda Geilo.
------------
Retrieving Results for Index 523: carutapera.
Closest hotel is Lidera.
------------
Retrieving Results for Index 524: bambari.
Closest hotel is Hotel-Restaurant BIBICH.
------------
Retrieving Results for Index 525: santa cruz cabralia.
Closest hotel is Baia Cabrália Hotel.
------------
Retrieving Results for Index 526: roma.
Closest hotel is Wingate By Wyndham Rome.
------------
Retrieving Results for Index 527: kitimat.
Closest hotel is Kingfish Westcoast Adventures Ltd. Kitimat Estuary Lodge B&B.
------------
Retrieving Results for Index 528: taksimo.
Closest hotel is Gostinitsa "Viktoriya".
------------
Retrieving Results for Index 529: zhigansk.
Closest hotel is Gostinichnyy Kompleks Aylgy.
------------
Retrieving Results for Index 530: san matias.
Closest hotel is HOTEL "SAN JOSE".
------------
Retrieving Results for Index 531: gifhorn.
Closest hotel is Morada Hotel Gifhorn.
------------
Retrieving Results for Index 532: celestu

In [6]:
#Show the hotel information for the cities listed
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,rikitea,61,PF,1586749115,77,-23.12,-134.97,77.70,18.05,Pension Maro'i
1,1,arman,100,RU,1586749115,94,59.70,150.17,25.32,4.07,NaN
2,2,port alfred,12,ZA,1586749116,79,-33.59,26.89,57.99,3.00,The Halyards Hotel
3,3,ushuaia,90,AR,1586748920,81,-54.80,-68.30,42.80,11.41,Albatros Hotel
4,4,mataura,99,NZ,1586748923,83,-46.19,168.86,45.00,11.99,Ellie's Villa
...,...,...,...,...,...,...,...,...,...,...,...
560,616,ninghai,0,CN,1586749276,31,29.29,121.42,64.99,15.66,Kaiyuan New Century Hotel
561,617,urazovka,0,RU,1586749276,87,55.40,45.62,30.07,12.50,"Krasnooktyabr'skoye, Gostinitsa"
562,618,tomatlan,22,MX,1586749277,52,19.93,-105.25,70.45,8.57,Norma franco
563,619,mackay,32,AU,1586749277,74,-21.15,149.20,82.40,16.11,International Lodge Motel


In [7]:
#Create a marker layer
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

#Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

#Add the layer to the heatmap
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))